In [1]:
from dotenv import dotenv_values
import os

env_vars = dotenv_values('../.env')
os.environ['OPENAI_API_KEY'] = env_vars.get('OPENAI_API_KEY')

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.chains import RetrievalQA

In [3]:
model = ChatOpenAI()

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator=".",
    chunk_size=600,
    chunk_overlap=100,
)
loader = UnstructuredFileLoader("./file/law.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

chain = RetrievalQA.from_chain_type(
    llm=model,
    chain_type="map_rerank",
    retriever=vectorstore.as_retriever(),
    )

In [4]:
chain.run("제5조는 무엇이야?")

/home/crysis/anaconda3/envs/llm/lib/python3.11/site-packages/langchain/chains/llm.py:349: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
